In [1]:
import pandas as pd
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np

## 讀取資料
- 讀取爬文和下載好的資料

In [2]:
# 讀取爬下來的文集
news100 = pd.read_excel('appledaily_politinews100.xlsx')
news100.head()

,category,content,title
0,政治,總統蔡英文卸任民進黨主席，昨在臉書po給黨員的一封信，信中說，民進黨會展開一場轟轟烈烈的檢討...,蔡英文要轟轟烈烈檢討民進黨 林濁水酸：民主國家沒聽過
1,政治,民進黨這次九合一選舉慘敗，蔡英文辭去黨主席一職，昨公布「給黨員的一封信」，內容提到，最該改變...,他列民進黨10大豬隊友 「不下架將賠上2020」
2,政治,2018年地方選舉國民黨拿下空前勝利，執政地區從6縣市竄升為15縣市，昨中常會上國民黨提出選...,選戰檢討報告出爐 國民黨：落選者「缺乏對外拓票企圖心」
3,政治,準高雄市長韓國瑜的愛女韓冰昨(28日)受邀上政論節目《54新觀點》，首度與民進黨台北市議員王...,不用跳愛河！韓冰邀替高雄拚觀光 王世堅靦腆：可以廢物利用
4,政治,點我直接觀看 被操弄的民調？《蘋果》數據解密,【民調解密3】林佳龍敗在紫爆 盧秀燕跌破眼鏡大勝


## 分詞/分句子，把文章轉換成Vector
- 清理文章，再分成句子
- 把句子轉換成count vector

In [3]:
# 選擇文章，把文章分成句子
docNum = 31
testDoc = news100['content'][docNum]
testTitle = news100['title'][docNum]

def splitSentences(document):
    #for sent in re.findall(u'[^!?。\.\!\?]+[!?。\.\!\?]?', paragraph, flags=re.U):
    for sent in re.findall(u'[^!?。\!\?]+[!?。\!\?]?', document, flags=re.U):
        yield sent

sentenceDoc = []
for sent in list(splitSentences(testDoc))[:-1]:
    sentenceDoc.append(' '.join(jieba.cut(sent)))

# 原本文章
testDoc

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\YB0002~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.772 seconds.
Prefix dict has been built succesfully.


'今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。中選會副主委陳朝建今證實曾遭駭客攻擊，不過計票網路是封閉系統，並非公開網路，無法由外界接取，因此沒有任何影響，境外網路干擾也並未影響投票進程。行政院發言人Kolas Yotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。不過，投票日當天開票進程緩慢，台北市選情更創下史上第一次開票長達9小時的紀錄，陳朝建說明，開票時間長是因為投票時間延後，計票不受駭客影響，開票也自然不受此影響。據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一般民眾並沒有發現特殊異樣。NCC及中華電信暫未對此做進一步說明。陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。陳朝建說，受到攻擊的官網，當日皆有隨時防衛，中華電信團隊24小時監控與防護作業；故當日遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊時，在負責網路維護中華電信的緊急應變下，並沒有受到影響。另外，Kolas表示，中選會的官網在選舉當天上午6時56分至7時01分期間，遭受大規模境外DDoS攻擊，而中選會的系統是封閉系統，防護非常嚴密，因此沒被攻擊，而當因中選會在選前即做好縝密的防護，當天其實沒因網路攻擊發生意外狀況，中選會的資安管理有成功防堵攻擊。Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。（林惟崧、鄭鴻達／台北報導）出版：1454更新：1942（新增行政院說法） \xa0想知道更多，一定要看……【獨家】投票日遭境外駭客攻擊\u3000中選會網站險癱瘓看了這則新聞的人，也看了……老里長險勝1票\u3000美女里長候選人要求驗票發祭品囉！\u3000韓總雞排逾千

In [4]:
# 分詞後的文章
sentenceDoc

['今年 九 合一 大選 投票 狀況 多 ， 據 了解 ， 中選會 官方 網站 在 投票 日當天 曾 遭到 來 自 境外 大規模 分散式 阻斷 服務 （ DD0oS ） 攻擊 ， 所幸 在 負責 網路 維護 中華電信 應變 下 ， 未進 一步 造成 影響 。',
 '中選會 副 主委 陳 朝 建今 證實 曾 遭駭客 攻擊 ， 不過 計票 網路 是 封閉 系統 ， 並 非公 開網 路 ， 無法 由 外界 接取 ， 因此 沒有 任何 影響 ， 境外 網路 干擾 也 並未 影響 投票 進程 。',
 '行政院 發言人 Kolas   Yotaka （ 谷辣斯 ・ 尤達卡 ） 表示 ， 中選會 在 選前 即 做好 縝密 防護 ， 因此 成功 防堵 攻擊 ， 接下 來 將針 對 有形 的 選務如 投票 排隊 問題 ， 以及 無形 的 選務 如資 安防 護進行 檢討 。',
 '不過 ， 投票 日當天 開票 進程 緩慢 ， 台北市 選情 更 創下史 上 第一次 開票 長 達 9 小時 的 紀錄 ， 陳 朝建 說明 ， 開票 時間 長 是 因為 投票 時間 延 後 ， 計票 不受 駭客 影響 ， 開票 也 自然 不受 此 影響 。',
 '據 內部 消息 指出 ， 投票 日當天 曾 出現 從 境外 進行 分散式 阻斷 服務 （ DDoS ） 攻擊 ， 企圖 癱瘓 中選會 網站 ， 不過 在 中華電信 、 中選會 緊急 處理 後 ， 阻斷 來 自國外 異常 流量 ， 在 短 時間 獲得 解決 ， 一般 民眾 並沒有 發現 特殊 異樣 。',
 'NCC 及 中華電信 暫未 對 此 做 進 一步 說明 。',
 '陳 朝建 表示 ， 受 攻擊 的 是 中 選會 官方 網站 ， 當日 開票 的 計票 網路 是 獨立 的 封閉 網路 ， 並 非公 開網 路 ， 無法 由 外界 接取 ， 完全 未有 任何 影響 ； 中華電信 團隊 也 是 24 小時 監控 與 防護 作業 ， 也 沒 有 受到 任何 影響 。',
 '陳 朝建 說 ， 受到 攻擊 的 官網 ， 當日 皆 有 隨時 防衛 ， 中華電信 團隊 24 小時 監控 與 防護 作業 ； 故當 日 遭到 來 自 境外 大規模 分散式 阻斷 服務 （ DD0oS ） 攻擊時 ， 在 負責 網路 維護 中華電信 的 緊急 應變 下 ， 並

In [5]:
# 把文章轉換成 count vector
vect = CountVectorizer()
doc_vector = vect.fit_transform(sentenceDoc)

In [6]:
# 計算 vector 之間的相似度
cs = cosine_similarity(doc_vector)
cs

array([[1.        , 0.29032258, 0.13116517, 0.18146655, 0.37387825,
        0.16064387, 0.23308662, 0.44901326, 0.18247917, 0.17255919],
       [0.29032258, 1.        , 0.13116517, 0.23331413, 0.17255919,
        0.        , 0.52444489, 0.1796053 , 0.27371875, 0.0862796 ],
       [0.13116517, 0.13116517, 1.        , 0.05270463, 0.11694107,
        0.        , 0.08885233, 0.06085806, 0.34780417, 0.46776428],
       [0.18146655, 0.23331413, 0.05270463, 1.        , 0.13867505,
        0.06454972, 0.2575611 , 0.09622504, 0.        , 0.04622502],
       [0.37387825, 0.17255919, 0.11694107, 0.13867505, 1.        ,
        0.07161149, 0.07792865, 0.26688026, 0.18302666, 0.17948718],
       [0.16064387, 0.        , 0.        , 0.06454972, 0.07161149,
        1.        , 0.07254763, 0.1490712 , 0.        , 0.        ],
       [0.23308662, 0.52444489, 0.08885233, 0.2575611 , 0.07792865,
        0.07254763, 1.        , 0.45962736, 0.22662355, 0.05195243],
       [0.44901326, 0.1796053 , 0.0608580

In [7]:
# 用 k-means clustering 把每個 vector 分成三群
km = KMeans(n_clusters=3, init='k-means++')
c = km.fit_predict(cs)
c

array([1, 0, 2, 0, 1, 1, 0, 1, 2, 2])

In [8]:
np_doc = np.array(sentenceDoc)
print(np_doc[c==0])

['中選會 副 主委 陳 朝 建今 證實 曾 遭駭客 攻擊 ， 不過 計票 網路 是 封閉 系統 ， 並 非公 開網 路 ， 無法 由 外界 接取 ， 因此 沒有 任何 影響 ， 境外 網路 干擾 也 並未 影響 投票 進程 。'
 '不過 ， 投票 日當天 開票 進程 緩慢 ， 台北市 選情 更 創下史 上 第一次 開票 長 達 9 小時 的 紀錄 ， 陳 朝建 說明 ， 開票 時間 長 是 因為 投票 時間 延 後 ， 計票 不受 駭客 影響 ， 開票 也 自然 不受 此 影響 。'
 '陳 朝建 表示 ， 受 攻擊 的 是 中 選會 官方 網站 ， 當日 開票 的 計票 網路 是 獨立 的 封閉 網路 ， 並 非公 開網 路 ， 無法 由 外界 接取 ， 完全 未有 任何 影響 ； 中華電信 團隊 也 是 24 小時 監控 與 防護 作業 ， 也 沒 有 受到 任何 影響 。']


## 分群
- 把每個句子分成群依照句子跟中心的相似度

In [9]:
# 把每個分群依照跟中心的相似度排列從大到小，找出最適合做摘要的句子
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
order_centroids

array([[6, 1, 3, 7, 0, 8, 4, 2, 9, 5],
       [0, 7, 4, 5, 6, 1, 8, 3, 9, 2],
       [2, 9, 8, 1, 0, 4, 6, 7, 3, 5]], dtype=int64)

In [10]:
# 找出每個群的前三個句子(i.e. 最重要/相似的句子)
centroid_top3 = km.cluster_centers_.argsort()[:, :6:-1]
centroid_top3

array([[6, 1, 3],
       [0, 7, 4],
       [2, 9, 8]], dtype=int64)

In [11]:
# 依照每個群print句子
clusterIndex = 0
for cluster in centroid_top3:
    print('Cluster: ' + str(clusterIndex))
    for sent in cluster:
        print('Sentence ' + str(sent), sentenceDoc[sent].replace(' ','').strip())
    clusterIndex += 1
    print('')

Cluster: 0
Sentence 6 陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。
Sentence 1 中選會副主委陳朝建今證實曾遭駭客攻擊，不過計票網路是封閉系統，並非公開網路，無法由外界接取，因此沒有任何影響，境外網路干擾也並未影響投票進程。
Sentence 3 不過，投票日當天開票進程緩慢，台北市選情更創下史上第一次開票長達9小時的紀錄，陳朝建說明，開票時間長是因為投票時間延後，計票不受駭客影響，開票也自然不受此影響。

Cluster: 1
Sentence 0 今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。
Sentence 7 陳朝建說，受到攻擊的官網，當日皆有隨時防衛，中華電信團隊24小時監控與防護作業；故當日遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊時，在負責網路維護中華電信的緊急應變下，並沒有受到影響。
Sentence 4 據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一般民眾並沒有發現特殊異樣。

Cluster: 2
Sentence 2 行政院發言人KolasYotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。
Sentence 9 Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。
Sentence 8 另外，Kolas表示，中選會的官網在選舉當天上午6時56分至7時01分期間，遭受大規模境外DDoS攻擊，而中選會的系統是封閉系統，防護非常嚴密，因此沒被攻擊，而當因中選會在選前即做好縝密的防護，當天其實沒因網路攻擊發生意外狀況，中選會的資安管理有成功防堵攻擊。



## 長文摘要

In [12]:
# 找出每個cluster最像中心的三個句子做摘要
for cluster in order_centroids:
    print('Sentence ' + str(cluster[0]) + '\n', sentenceDoc[cluster[0]].replace(' ','').strip())

Sentence 6
 陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。
Sentence 0
 今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。
Sentence 2
 行政院發言人KolasYotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。


**會提高摘要效果的改進**
1. 更好的分詞/分句子會有更好的效果/摘要
2. 更進階的方法計算句子的重要性

## Cluster Graph Attempt

In [13]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3'}

#set up cluster names using a dict
cluster_names = {0: 'Cluster 0', 
                 1: 'Cluster 1', 
                 2: 'Cluster 2'}

In [14]:
order_centroids = km.cluster_centers_.argsort()[:, :6:-1]
order_centroids

array([[6, 1, 3],
       [0, 7, 4],
       [2, 9, 8]], dtype=int64)